# Check Output distance for perturbed samples

In this notebook, we perturb samples with Gaussian noise, and we check the improvements brought by stability training.

We import useful librairies

In [1]:
import sys, os 
sys.path.append(os.getcwd()+'/../src')
sys.path.append(os.getcwd()+'/../data')

import numpy
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

import architectures
import cifar10
import utils
import callbacks

We import CIFAR Data

In [2]:
X_test, y_test_lab = cifar10.load_cifar(dataset='testing', path=os.getcwd()+'/../data/cifar')
X_test = numpy.reshape(X_test, [X_test.shape[0], -1])
X_test = X_test/255.
X_test = X_test.astype(numpy.float32)
y_test = utils.labels_to_categoricals(y_test_lab)

We perturb samples

In [3]:
pert = 0.05*numpy.random.randn(10000, 3072)
X_test_p = X_test+pert

We load regular network and stabilized one (1st configuration):

In [6]:
stab_net = architectures.CifarNet(path_to_logs=os.getcwd()+'/Logs')
stab_net.load('Networks/StabilityTuned3')
class_net = architectures.CifarNet(path_to_logs=os.getcwd()+'/Logs')
class_net.load('Networks/RegularTuned')

We compute mean KL-Divergence for regularly trained network

In [8]:
out1 = class_net.evaluate_output(X_test)
out2 = class_net.evaluate_output(X_test_p)
diff = -out1*numpy.log(out2)
print(diff.mean())

0.0536227


We compute mean KLL-Divergence for stability trained network

In [7]:
out1 = stab_net.evaluate_output(X_test)
out2 = stab_net.evaluate_output(X_test_p)
diff = -out1*numpy.log(out2)
print(diff.mean())

0.038553


As expected, the KL-Div is lower for stability training.